# Cleaning Astronomical Datasets

I have posed two problems for you to work on in this hands-on exercise.  

- Concept drift: Do the training and test set distributions differ?
- Find mislabeled examples in the labeled ZTF data provided

More information for each is provided below.  You may want or need to cut and paste code from your other notebooks.  But first...


### 0a. Imports

These are all the imports that will be used in this notebook.  All should be available in the DSFP conda environment.

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
# You can add anything you need as you work

### 0b. Data Location

We will use the same data from the Day 2 clustering exercise (see [that notebook](https://github.com/LSSTC-DSFP/LSSTC-DSFP-Sessions/blob/master/Session7/Day2/Clustering-Astronomical-Sources.ipynb) to download the data).

Please specify paths for the following:

In [2]:
F_META = '../Day2/dsfp_ztf_meta.npy'
F_FEATS = '../Day2/dsfp_ztf_feats.npy'
D_STAMPS = '../Day2/dsfp_ztf_png_stamps'

### 0c. Load Data

In [4]:
meta_np = np.load(F_META)
feats_np = np.load(F_FEATS)

COL_NAMES = ['diffmaglim', 'magpsf', 'sigmapsf', 'chipsf', 'magap', 'sigmagap',
             'distnr', 'magnr', 'sigmagnr', 'chinr', 'sharpnr', 'sky',
             'magdiff', 'fwhm', 'classtar', 'mindtoedge', 'magfromlim', 'seeratio',
             'aimage', 'bimage', 'aimagerat', 'bimagerat', 'elong', 'nneg',
             'nbad', 'ssdistnr', 'ssmagnr', 'sumrat', 'magapbig', 'sigmagapbig',
             'ndethist', 'ncovhist', 'jdstarthist', 'jdendhist', 'scorr', 'label']

# NOTE FROM Umaa: I've decided to eliminate the following features. Dropping them.
#
COL_TO_DROP = ['ndethist', 'ncovhist', 'jdstarthist', 'jdendhist', 
               'distnr', 'magnr', 'sigmagnr', 'chinr', 'sharpnr', 
               'classtar', 'ssdistnr', 'ssmagnr', 'aimagerat', 'bimagerat', 
               'magapbig', 'sigmagapbig', 'scorr']
feats_df = pd.DataFrame(data=feats_np, index=meta_np['candid'], columns=COL_NAMES)
print("There are {} columns left.".format(len(feats_df.columns)))
print("They are: {}".format(list(feats_df.columns)))
feats_df.drop(columns=COL_TO_DROP, inplace=True) 
feats_df.describe()

There are 36 columns left.
They are: ['diffmaglim', 'magpsf', 'sigmapsf', 'chipsf', 'magap', 'sigmagap', 'distnr', 'magnr', 'sigmagnr', 'chinr', 'sharpnr', 'sky', 'magdiff', 'fwhm', 'classtar', 'mindtoedge', 'magfromlim', 'seeratio', 'aimage', 'bimage', 'aimagerat', 'bimagerat', 'elong', 'nneg', 'nbad', 'ssdistnr', 'ssmagnr', 'sumrat', 'magapbig', 'sigmagapbig', 'ndethist', 'ncovhist', 'jdstarthist', 'jdendhist', 'scorr', 'label']


,diffmaglim,magpsf,sigmapsf,chipsf,magap,sigmagap,sky,magdiff,fwhm,mindtoedge,magfromlim,seeratio,aimage,bimage,elong,nneg,nbad,sumrat,label
count,3190.000000,3190.000000,3190.000000,3190.000000,3190.000000,3190.000000,3190.000000,3190.000000,3190.000000,3190.000000,3190.000000,3190.000000,3190.000000,3190.000000,3190.000000,3190.000000,3190.000000,3190.000000,3190.000000
mean,20.151915,19.038766,0.114721,29.171392,18.882236,0.133377,1.759273,-0.156530,5.762724,535.662895,1.269680,-19.033703,1.447092,1.121355,1.277802,3.702508,0.827586,0.942922,0.263636
std,0.600403,1.182225,0.059628,184.693580,1.146221,0.090430,11.922192,0.361237,8.082798,358.128782,1.056042,140.283975,1.359441,1.081769,0.233597,2.640393,1.852272,0.093331,0.440673
min,16.387886,11.861366,0.000729,0.145480,11.904000,0.000900,-62.988770,-0.999049,-0.840000,10.071900,-0.689344,-999.000000,0.289000,0.256000,1.000000,0.000000,0.000000,0.400054,0.000000
25%,19.816405,18.404491,0.066442,1.752059,18.243625,0.062925,-0.078796,-0.387930,2.080000,233.525776,0.485384,0.392570,0.771250,0.667000,1.096764,2.000000,0.000000,0.921823,0.000000
50%,20.221339,19.230843,0.111248,3.588183,19.093750,0.121900,0.326771,-0.088569,3.170000,475.807602,1.019777,0.794723,1.018000,0.841000,1.201702,3.000000,0.000000,0.990973,0.000000
75%,20.608219,19.917913,0.167752,11.765462,19.744100,0.185475,0.891043,0.057932,6.200000,782.824844,1.860215,1.171276,1.726750,1.268000,1.395918,5.000000,0.000000,1.000000,1.000000
max,21.169477,21.312767,0.217133,7037.184570,21.160299,0.733300,321.040314,0.971744,118.790001,1507.777222,8.407010,116.662155,23.407000,15.154000,2.000000,13.000000,7.000000,1.000000,1.000000


## 1. Concept Drift

In the last exercise, you created a training and test set for the purposes of building a classifier.  The goal of this exercise is to note any changes in the feature distributions for these two sets.

Per feature, can you:

- plot test vs. train distributions for both real and bogus, and note areas that does not overlap
- quantiatively measure this using [Kullback-Leibler divergence](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.entropy.html), and print or plot the scores for all features.

Which feature exhibits the most drift between train and test?

In [ ]:
plt.plot(test_[])

## 2. Finding Mislabeled Examples

This task ties together the work you did for the unsupervised and supervised exercises.  Here's how to get started.

1. Cluster the entire labeled set provided to you.  How you choose to do the clustering is up to you.  I would recommend getting your clustering results into a two dimensional space that you can plot, but this is not strictly necessary.

2. Apply the labels to the clusters you've created and plot them.  If you're working in a >=3-dimensional space, find a way to print the candids in each cluster.  You can sort the list by examples that are closet to the centroids, and print their associated labels.

3. Look at some postage stamps of examples you suspect are mislabeled.  Can you devise a simple way to identify a set of mislabeled examples?  Can you come up with an estimate of how many examples are mislabeled?